# Import Necessary Libraries

In [1]:
# Import all the necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
import re
import json
import mysql.connector

# Reusable Functions

1. Initial Setup

In [2]:
# Set the file path to save the tour details in a structured JSON format
tour_details_file_path = 'agoda_tour_details.json'

def setup_driver():
    """Function to set up the Chrome Driver with custom options."""
    options = Options()
    options.add_argument("user-agent=Mozilla/5.0")
    
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(10)  # Implicit wait for element presence
    driver.set_script_timeout(120)  # Timeout for asynchronous scripts
    driver.set_page_load_timeout(10)  # Page load timeout
    
    return driver

def wait_for_clickable_element(driver, selector, timeout=20):
    """Wait for an element to be present and clickable on the page."""

    clickable_elements = []
    # Wait for the elements to be present in the DOM
    elements = WebDriverWait(driver, timeout).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, selector))
    )
    # Wait for elements to be clickable
    for element in elements:
        if WebDriverWait(driver, timeout).until(EC.element_to_be_clickable((element))):
            clickable_elements.append(element)
    return clickable_elements

2. Scrape Agoda.com for tours in San Francisco that last for either 1-4 hours or more than 4 hours

In [ ]:
def navigate_to_agoda(driver):
    """Navigates to the Agoda website and performs a search for activities in San Francisco.
       Filters for tours that last for either 1-4 hours or more than 4 hours"""

    # Initiate a list to store page sources
    page_urls = []

    # Navigate to agoda.com
    driver.get('https://www.agoda.com/')

    # Navigate to the "Activities" tab
    activities_tab = wait_for_clickable_element(driver, "li#tab-activities-tab")
    activities_tab[0].click()

    # Search for "San Francisco"
    search_input = wait_for_clickable_element(driver, "input#activities-search-input")
    search_input[0].send_keys('San Francisco') # Replace with place of interest

    # Select "San Francisco (CA), United States" autocomplete suggestion
    suggestion_li = wait_for_clickable_element(driver, "li.Suggestion.LocationSearch.Suggestion__categoryName[data-objectid='13801']")
    suggestion_li[0].click()

    # Search for activities
    search_activities = wait_for_clickable_element(driver, "button[data-element-name='activities-search-button']")
    search_activities[0].click()
    # Add a 20 sec delay
    time.sleep(20)

    # Filter for "Tours"
    tours = driver.find_element(By.XPATH, "//p[contains(text(), 'Tours')]")
    tours.click()
    # Add a 20 sec delay
    time.sleep(20)

    # Filter for "1-4 hours" duration activities
    duration_1to4h = driver.find_element(By.XPATH, "//input[@aria-label='1-4 hours']")
    if not duration_1to4h.is_selected():
        duration_1to4h.click()
        
    # Add a 20 sec delay
    time.sleep(20)

    page_urls.append(driver.current_url)

    # Uncheck the "1-4 hours" duration filter if checked
    if duration_1to4h.is_selected():
        duration_1to4h.click()

    # Filter for "More than 4 hours" duration activities
    duration_morethan4h = driver.find_element(By.XPATH, "//input[@aria-label='More than 4 hours']")
    if not duration_morethan4h.is_selected():
        duration_morethan4h.click()

    # Add a 20 sec delay
    time.sleep(20)

    page_urls.append(driver.current_url)
    print(f'Tour Source URLs: {page_urls}\n')
    
    return page_urls

def get_tours(driver, page_urls):
    """Gets links to all the tours in San Francisco which last for either 1-4 hours or more than 4 hours."""

    # Initialize a list to hold the links to all the tours
    tour_links = []

    for page_url in page_urls:
        driver.get(page_url)
        # Add a 20 sec delay
        time.sleep(20)

        # Initialize a list to hold the links
        activity_links = []

        # Get the page count
        page_count_span = driver.find_element(By.CSS_SELECTOR, "span[data-element-name='activities-pagination-page-count'] > span")
        page_count = page_count_span.text.strip()

        # Extract the maximum page number
        match = re.search(r'Page \d+ of (\d+)', page_count)
        max_page_number = int(match.group(1))

        # Keep track of whether to continue to the next page
        continue_to_next_page = True

        while continue_to_next_page:
            # Add a 20 sec delay
            time.sleep(20)
            # Wait for the activities container to be clickable
            activities_container = wait_for_clickable_element(driver, "div[class='sc-kOPcWz kNxCTa']")

            # Collect all the links from the current page
            for container in activities_container:
                # Find all 'a' elements within this container
                activities = container.find_elements(By.CSS_SELECTOR, 'a[data-element-name="activities-card-content"]')
                for a_element in activities:
                    href = a_element.get_attribute('href')
                    activity_links.append(href)

            # Check if there is a next page
            if(max_page_number != 1):
                    driver.get(f'{page_url}&page={max_page_number}')
                    max_page_number = max_page_number - 1
                    # Add a 20 sec delay
                    time.sleep(20)
            else:
                continue_to_next_page = False

        tour_links.append(activity_links)

    # Creating a dictionary with specified keys
    activities_dict = {
        "1-4 hours": tour_links[0],
        "More than 4 hours": tour_links[1]
    }

    # Save the links to a JSON file
    file_path = 'agoda_tour_links.json'

    with open(file_path, 'w') as file:
        json.dump(activities_dict, file, indent=4)
    
    print(f'Individual Activity URLs: {activities_dict}')

    return activities_dict

3. Extract tour details such as overview, duration, package options, prices, #reviews, cancellation and pick-up information among others.

In [ ]:
# Function to scrape tour details
def scrape_tour_details(driver, tours):
    tour_details = []
    for duration_type, links in tours.items():
        for link in links:
            try:
                driver.get(link)
                time.sleep(20)

                # Get details card
                try:
                    details_card = driver.find_element(By.CSS_SELECTOR, "div[data-element-name='details-card']")
                except NoSuchElementException:
                    print("Details card not found for:", link)
                    continue

                # Initialize elements
                title_element = None
                rating_element = None
                reviews_element = None
                duration_element = None
                pickup_offered_element = None
                free_cancellation_element = None
                overview_element = None

                # Extract tour details
                try:
                    title_element = details_card.find_element(By.XPATH, ".//h2[@data-element-name='details-title']")
                except NoSuchElementException:
                    pass

                try:
                    rating_element = details_card.find_element(By.XPATH, ".//span[@class='sc-dhKdcB sc-kpDqfm bkykib hWIusU']")
                except NoSuchElementException:
                    pass

                try:
                    reviews_element = details_card.find_element(By.XPATH, ".//p[@class='sc-dhKdcB sc-kpDqfm eVYCpt bgDSRY']")
                except NoSuchElementException:
                    pass

                try:
                    duration_element = details_card.find_element(By.XPATH, ".//div[@data-element-name='activities-details-duration']")
                except NoSuchElementException:
                    pass

                try:
                    pickup_offered_element = details_card.find_element(By.XPATH, ".//div[@data-element-name='activities-details-pickUp']")
                except NoSuchElementException:
                    pass

                try:
                    free_cancellation_element = details_card.find_element(By.XPATH, ".//div[@data-element-name='activities-details-freeCancellation']")
                except NoSuchElementException:
                    pass

                try:
                    overview_element = driver.find_element(By.XPATH, "//p[@data-element-name='overview-description']")
                except NoSuchElementException:
                    pass

                duration_text = duration_element.text.strip() if duration_element else None
                duration = None

                if duration_text:
                    # Check if duration contains "day" or "days"
                    if "day" in duration_text:
                        # Extract the number of days using regular expression
                        days = float(re.search(r'\d+(\.\d+)?', duration_text).group())
                        # Convert days to hours
                        duration = days * 24
                    # Check if duration contains "hour" or "hours"
                    elif "hour" in duration_text:
                        # Extract the number of hours using regular expression
                        hours = float(re.search(r'\d+(\.\d+)?', duration_text).group())
                        duration = hours
                    else:
                        # If neither "day" nor "hour" is present, return None or handle as needed
                        duration = None

                details = {
                    'tour_details': {
                        'title': title_element.text.strip() if title_element else None,
                        'rating': float(rating_element.text.strip()) if rating_element else None,
                        'n_reviews': int(''.join(filter(str.isdigit, reviews_element.text.strip()))) if reviews_element else 0,
                        'duration': float(duration) if duration else None,
                        'is_pickup_offered': bool(pickup_offered_element),
                        'is_risk_free_cancellation': bool(free_cancellation_element),
                        'overview': overview_element.text.strip() if overview_element else None,
                        'starting_price_usd': None,
                        'duration_type': duration_type
                    }
                }

                # Extract package options for the tour
                package_options = extract_package_options(driver, link)
                details['package_options'] = package_options
                details['tour_details']['starting_price_usd'] = float(package_options[0]['final_price_usd']) if package_options else None

                tour_details.append(details)

                with open(tour_details_file_path, 'w') as file:
                    json.dump(tour_details, file, indent=4)

            except Exception as e:
                print(f"Error processing tour: {link}, Error: {e}")

    return tour_details

# Function to extract package options for a tour
def extract_package_options(driver, source_link):
    # Initiate a list to store the package options
    package_options = []
    # Find the data-chip elements
    date_chip_elements = driver.find_elements(By.XPATH, "//button[@data-element-name='date-chip-element']")

    # Iterate over each date chip element
    for date_chip in date_chip_elements:

        # Check if the button is disabled
        if not date_chip.get_attribute("disabled"):
            # Execute JavaScript to click on the chip element
            driver.execute_script("arguments[0].click();", date_chip)
            time.sleep(20)
            # Wait for the details-available-package-option-container to appear
            details_container = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, "//div[@data-element-name='details-available-package-option-container']"))
            )

            available_options = details_container.find_elements(By.XPATH, "//div[@data-element-name='activities-available-option-card']")

            # Check if package options are present
            if available_options:
                # Extract package options
                for option in available_options:

                    # Initialize elements
                    detail_title_element = None
                    original_price_element = None
                    final_price_element = None

                    try:
                        detail_title_element = option.find_element(By.XPATH, ".//h3[@data-element-name='available-option-card-detail-title']")
                    except NoSuchElementException:
                        pass

                    try:
                        original_price_element = option.find_element(By.XPATH, ".//span[@data-element-name='activities-original-price']")
                    except NoSuchElementException:
                        pass

                    try:
                        final_price_element = option.find_element(By.XPATH, ".//span[@class='sc-dhKdcB sc-kpDqfm bKaiWl dWWROR']")
                    except NoSuchElementException:
                        pass

                    package_details = {
                        'title': detail_title_element.text if detail_title_element else None,
                        'original_price_usd': float(original_price_element.text.split()[-1].replace(',', '')) if original_price_element else None,
                        'final_price_usd': float(final_price_element.text.strip().replace(',', '')) if final_price_element else None
                    }
                    package_options.append(package_details)

                    # Check whether only the 'starting from' (final_price_usd) price is mentioned
                    if package_details['original_price_usd'] is None and package_details['final_price_usd'] is not None:
                        package_details['original_price_usd'] = package_details['final_price_usd']

                # Break the loop once options are extracted
                break
            else:
                print(f"No package options found for this tour: {source_link}.")

    return package_options

4. Database and Tables Creation for Tours and Package Options

In [ ]:
def create_datasets():
    """Create a mySQL database and tables to store the tours and their respective package options"""

    try:
        # Initiate a variable to flag once the tables are created successfully
        is_successful = False

        # Connect to MySQL
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="<INSERT PASSWORD>" # Insert password here.
        )
        cursor = conn.cursor()

        # Create a new database and select it for use
        cursor.execute("CREATE DATABASE IF NOT EXISTS agoda_sf_tours")
        cursor.execute("USE agoda_sf_tours")

        # Create 'tour' table
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS tour (
            tour_id INT AUTO_INCREMENT PRIMARY KEY,
            title VARCHAR(100),
            rating DECIMAL(3, 1),
            n_reviews INT,
            duration DECIMAL(3, 1),
            is_pickup_offered BOOLEAN,
            is_risk_free_cancellation BOOLEAN,
            overview TEXT,
            starting_price_usd DECIMAL(10, 2),
            duration_type VARCHAR(20)
        )
        ''')

        # Create 'package_option' table
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS package_option (
            id INT AUTO_INCREMENT PRIMARY KEY,
            tour_id INT,
            title TEXT,
            original_price_usd DECIMAL(10, 2),
            final_price_usd DECIMAL(10, 2),
            FOREIGN KEY (tour_id) REFERENCES Tours (tour_id)
        )
        ''')

        # Load tour details from JSON file
        with open(tour_details_file_path, 'r') as json_file:
            json_data = json.load(json_file)

        tours_values = []
        package_options_values = []

        # Prepare data for insertion
        for tour_count, tour in enumerate(json_data, start=1):
            tour_data = tour["tour_details"]
            tours_values.append((
                tour_data["title"], tour_data["rating"], tour_data["n_reviews"], tour_data["duration"],
                tour_data["is_pickup_offered"], tour_data["is_risk_free_cancellation"], tour_data["overview"],
                tour_data["starting_price_usd"], tour_data["duration_type"]
            ))

            for package_option in tour["package_options"]:
                package_options_values.append((
                    tour_count, package_option["title"], package_option["original_price_usd"],
                    package_option["final_price_usd"]
                ))

        # Insert data into 'tour' table
        tours_insert_query = """
            INSERT INTO tour (title, rating, n_reviews, duration, is_pickup_offered, is_risk_free_cancellation, overview, starting_price_usd, duration_type)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.executemany(tours_insert_query, tours_values)

        if cursor.rowcount != len(tours_values):
            raise Exception("Failed to insert all tour data")
        else:
            # Insert data into 'package_option' table
            package_options_insert_query = """
                INSERT INTO package_option (tour_id, title, original_price_usd, final_price_usd)
                VALUES (%s, %s, %s, %s)
            """
            cursor.executemany(package_options_insert_query, package_options_values)

            if cursor.rowcount != len(package_options_values):
                raise Exception("Failed to insert all package_option data")
            else:
                is_successful = True

        # Commit changes and close connection
        conn.commit()

    except Exception as e:
        print(f"Error: {e}")
        # Drop rows from the tables if there's an error
        cursor.execute("DELETE FROM package_option")
        cursor.execute("DELETE FROM tour")

        conn.rollback()
        conn.close()

    finally:
        if conn.is_connected():
            conn.close()
        
        return is_successful

# Main Function

In [1]:
if __name__ == "__main__":
    driver = setup_driver()
    try:
        page_urls = navigate_to_agoda(driver)
        # Hardcording the URLs because I noticed that the website was conducting A/B testing when I was half way through the project.
        page_urls = ['https://www.agoda.com/activities/search?cityId=13801&currency=USD&category=11&duration=2', 'https://www.agoda.com/activities/search?cityId=13801&currency=USD&category=11&duration=3']
        tours = get_tours(driver, page_urls)
        tour_details = scrape_tour_details(driver, tours)

        if tour_details:
            # Create a database with tables for Tours and Package Options
            is_successful = create_datasets()

            if is_successful:
                print('Bulk insert completed successfully.')
    finally:
        driver.quit()

Tour Source URLs: ['https://www.agoda.com/activities/search?cityId=13801&currency=USD&category=11&duration=2', 'https://www.agoda.com/activities/search?cityId=13801&currency=USD&category=11&duration=3']
Individual Activity URLs: {'1-4 hours': ['https://www.agoda.com/activities/detail?activityId=951091&cityId=13801&ds=skTFEjW4akK1sAVt&currency=USD&category=11&duration=2', 'https://www.agoda.com/activities/detail?activityId=973239&cityId=13801&ds=skTFEjW4akK1sAVt&currency=USD&category=11&duration=2', 'https://www.agoda.com/activities/detail?activityId=1044326&cityId=13801&ds=skTFEjW4akK1sAVt&currency=USD&category=11&duration=2', 'https://www.agoda.com/activities/detail?activityId=1126129&cityId=13801&ds=skTFEjW4akK1sAVt&currency=USD&category=11&duration=2', 'https://www.agoda.com/activities/detail?activityId=1108182&cityId=13801&ds=skTFEjW4akK1sAVt&currency=USD&category=11&duration=2', 'https://www.agoda.com/activities/detail?activityId=1042244&cityId=13801&ds=skTFEjW4akK1sAVt&currency=U